In [ ]:
# Importing data
from pathlib import Path
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [52]:
import pandas as pd 
# weather_df = pd.read_csv("./drive/MyDrive/Data/weather.csv")
weather_aggregate_df = pd.read_csv("./drive/MyDrive/Data/weather_aggregate.csv")

In [ ]:
# weather_aggregate_df.head()
# The goal is to predict the temperature for a few days and see how the model is working
# Based on some information that we have for January

# features to consider = DEWP, TEMP, WDSP, MAX, MIN, ELEV(M)

In [53]:
weather_aggregate_df

,USAF,WBAN_x,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY,WBAN_y,STATION NAME,CTRY,STATE,LAT,LON,ELEV(M),LBL,YEARMODA
0,690150,93121.0,2000.0,1.0,51.20,26.60,4.30,66.20,39.9,17.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-01-01
1,690150,93121.0,2000.0,2.0,53.60,32.70,6.90,66.20,44.6,15.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-02-01
2,690150,93121.0,2000.0,3.0,61.70,29.90,6.50,73.40,46.9,16.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-03-01
3,690150,93121.0,2000.0,4.0,70.55,34.35,8.15,84.55,57.1,15.5,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-04-01
4,690150,93121.0,2000.0,5.0,79.00,42.50,9.70,93.90,64.4,16.0,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,625.1,"TWENTY NINE PALMS, CA, US<br>Elevation: 625.1 m",2000-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142737,749045,99999.0,2018.0,8.0,82.30,72.70,3.40,94.30,73.9,16.0,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-08-01
142738,749045,99999.0,2018.0,9.0,82.10,73.55,3.60,93.95,75.0,15.5,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-09-01
142739,749045,99999.0,2018.0,10.0,80.00,70.00,4.60,91.60,71.6,16.0,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-10-01
142740,749045,99999.0,2018.0,11.0,73.45,65.65,3.65,87.45,65.3,14.5,99999,SEBRING RGNL,US,FL,27.456,-81.342,19.2,"SEBRING RGNL, FL, US<br>Elevation: 19.2 m",2018-11-01


In [54]:
# Filtering only one month
# jan_data = weather_aggregate_df[weather_aggregate_df['MONTH'] == 1.0]
# jan_data

In [55]:
# Filter data only for Delaware, because otherwise it is too much data to process and there is not enough RAM 
#de_data = weather_df[(weather_df['STATE'] == 'DE')] # |  (weather_df['STATE'] == 'RI')]
df_data = weather_aggregate_df.drop(columns=["USAF","WBAN_x","YEARMODA","MAX","MIN","WBAN_y","DEWP","WDSP","STATE", "LBL", "STATION NAME","CTRY","ELEV(M)"])
df_data

,YEAR,MONTH,TEMP,DAY,LAT,LON
0,2000.0,1.0,51.20,17.0,34.300,-116.167
1,2000.0,2.0,53.60,15.0,34.300,-116.167
2,2000.0,3.0,61.70,16.0,34.300,-116.167
3,2000.0,4.0,70.55,15.5,34.300,-116.167
4,2000.0,5.0,79.00,16.0,34.300,-116.167
...,...,...,...,...,...,...
142737,2018.0,8.0,82.30,16.0,27.456,-81.342
142738,2018.0,9.0,82.10,15.5,27.456,-81.342
142739,2018.0,10.0,80.00,16.0,27.456,-81.342
142740,2018.0,11.0,73.45,14.5,27.456,-81.342


In [56]:
df_data.dtypes

YEAR     float64
MONTH    float64
TEMP     float64
DAY      float64
LAT      float64
LON      float64
dtype: object

In [ ]:
#de_data['YEARMODA'] =  pd.to_datetime(de_data['YEARMODA'])

In [57]:
# Testing and training data
# Convert categorical data to numeric with `pd.get_dummies`
with_dummies = pd.get_dummies(df_data)
with_dummies

,YEAR,MONTH,TEMP,DAY,LAT,LON
0,2000.0,1.0,51.20,17.0,34.300,-116.167
1,2000.0,2.0,53.60,15.0,34.300,-116.167
2,2000.0,3.0,61.70,16.0,34.300,-116.167
3,2000.0,4.0,70.55,15.5,34.300,-116.167
4,2000.0,5.0,79.00,16.0,34.300,-116.167
...,...,...,...,...,...,...
142737,2018.0,8.0,82.30,16.0,27.456,-81.342
142738,2018.0,9.0,82.10,15.5,27.456,-81.342
142739,2018.0,10.0,80.00,16.0,27.456,-81.342
142740,2018.0,11.0,73.45,14.5,27.456,-81.342


In [58]:
# Split the preprocessed data into our features and target arrays
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# drop the whole 2005 year data
X = with_dummies.drop(['TEMP'], axis=1).values
y = with_dummies[['TEMP']].values

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [59]:
X_train, X_test, y_train, y_test

(array([[ 2.01400e+03,  8.00000e+00,  1.60000e+01,  4.12430e+01,
         -7.69220e+01],
        [ 2.00500e+03,  1.00000e+00,  1.60000e+01,  4.07210e+01,
         -1.14036e+02],
        [ 2.01500e+03,  1.10000e+01,  1.55000e+01,  3.96740e+01,
         -7.56060e+01],
        ...,
        [ 2.01100e+03,  1.00000e+01,  1.60000e+01,  3.42010e+01,
         -1.19207e+02],
        [ 2.01700e+03,  5.00000e+00,  1.60000e+01,  4.07940e+01,
         -7.31020e+01],
        [ 2.01800e+03,  6.00000e+00,  1.55000e+01,  3.10500e+01,
         -9.31830e+01]]),
 array([[2000.   ,   12.   ,   16.   ,   37.512,  -77.323],
        [2000.   ,    7.   ,   16.   ,   32.734, -117.183],
        [2009.   ,    6.   ,   15.5  ,   41.787, -111.853],
        ...,
        [2000.   ,    8.   ,   16.   ,   40.918,  -81.444],
        [2006.   ,    3.   ,   16.   ,   35.056,  -89.987],
        [2002.   ,    6.   ,   15.5  ,   36.119,  -86.689]]),
 array([[69.8 ],
        [30.8 ],
        [52.25],
        ...,
        [62.

In [60]:
# # Train the Logistic Regression model on the unscaled data and print the model score
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# print(f'Model: {clf}')
# print(f'Training Score: {clf.score(X_train, y_train)}')
# print(f'Testing Score: {clf.score(X_test, y_test)}')




In [61]:
# # Train a Random Forest Classifier model and print the model score
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier()
# clf.fit(X_train, y_train)
# print(f'Model: {clf}')
# print(f'Training Score: {clf.score(X_train, y_train)}')
# print(f'Testing Score: {clf.score(X_test, y_test)}')

In [62]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from matplotlib import pyplot as plt
def test_model(model, data):
    X_train,X_test, y_train, y_test = data
    reg = model.fit(X_train, y_train)
    print(f'Model: {type(reg).__name__}')
    print(f'Train score: {reg.score(X_train, y_train)}')
    print(f'Test Score: {reg.score(X_test, y_test)}\n')
    plt.show() 

In [63]:
# Scale the data
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)
data = [X_train,X_test, y_train.ravel(), y_test.ravel()]

In [77]:
# test_model(LinearRegression(), data)
# test_model(KNeighborsRegressor(), data)
# test_model(RandomForestRegressor(), data)
test_model(ExtraTreesRegressor(), data)
# test_model(AdaBoostRegressor(), data)
# test_model(SVR(C=1.0, epsilon=0.2), data)

Model: ExtraTreesRegressor
Train score: 0.9999999864054354
Test Score: 0.9741343663107811



In [78]:
import pickle
regressor = ExtraTreesRegressor()
regressor.fit(X_train, y_train.ravel())
# Saving model to disk
pickle.dump(regressor, open('ET.pkl','wb'))



In [79]:
# Loading model to compare the results
model = pickle.load( open("ET.pkl",'rb'))
print(model.predict([[2019.0,	7.0,	15.0,	34.300,	-116.167]]))

[92.8055]
